In [ ]:
import os
import pandas as pd
import numpy as np

from pathlib import Path

path = Path('../input/tabular-playground-series-may-2021')

In [ ]:
os.listdir(path)

In [ ]:
train = pd.read_csv(path / 'train.csv')
test = pd.read_csv(path / 'test.csv')
sub = pd.read_csv(path / 'sample_submission.csv')

In [ ]:
train.head()

In [ ]:
targets = train['target']
features = train.drop(columns = ['id','target'])

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
targets = le.fit_transform(targets)
targets

In [ ]:
le.classes_

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(features.values, targets, test_size = 0.1, shuffle = True, random_state = 44)

In [ ]:
import optuna.integration.lightgbm as lgb
dtrain = lgb.Dataset(xtrain, label = ytrain)
dtest = lgb.Dataset(xtest, label = ytest)
params = {'objective' : 'multiclass', 'num_class' : 4,  'metric' : 'multi_logloss', 'verbosity' : -1, 'boosting_type' : 'gbdt'}
model = lgb.train(params, dtrain, valid_sets = [dtest], verbose_eval = 100, early_stopping_rounds = 100)

In [ ]:
params = model.params
params

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(**params)

In [ ]:
model.fit(xtrain, ytrain, eval_set = ((xtest, ytest)), early_stopping_rounds = 50, verbose = 0)

In [ ]:
sub.head()

In [ ]:
sub['id'] = test['id']

In [ ]:
test = test.drop(columns = ['id'])

In [ ]:
preds = model.predict_proba(test)

In [ ]:
preds

In [ ]:
sub['Class_1']=preds[:,0]
sub['Class_2']=preds[:,1]
sub['Class_3']=preds[:,2]
sub['Class_4']=preds[:,3]
sub.head()

In [ ]:
sub.to_csv('Submission.csv', index = False)
sub.head()

Please Upvote this kernel if you like it!